# Uploading Annotations

In [3]:
%load_ext autoreload
%autoreload 2
import remo
import os
import pandas as pd


    (\(\ 
    (>':') Remo server is running: v0.3.9-15-g9b08b86a
                


We start off by creating a dataset and populating it with some images

In [ ]:
urls = ['https://remo-scripts.s3-eu-west-1.amazonaws.com/open_images_sample_dataset.zip']
my_dataset = remo.create_dataset(name = 'D1', urls = urls)


## Annotation set

**Let's now create an empty annotation set with a predetermined list of classes**

In [5]:
my_classes = ['Airplane', 'Clothing', 'Dog', 'Fashion accessory', 'Food', 'Footwear', 'Fruit', 'Human arm', 
         'Human body', 'Human hand', 'Human leg', 'Mammal', 'Man', 'Person', 'Salad', 'Sports equipment', 'Trousers', 'Woman']

annotation_set = my_dataset.create_annotation_set(annotation_task = 'Object detection',
                                          name = 'Objects',
                                          classes = my_classes)

**We can easily retrieve all the annotation sets of a dataset**

In [6]:
annotation_set = my_dataset.annotation_sets()[0]
annotation_set

Annotation set 16 - 'Objects', task: Object detection, #classes: 18

## Add annotations from supported CSV file format

**Let's add some annotations from a CSV file**

In this example, annotations are stored in a CSV file in a format already supported by Remo. Class labels are encoded using [GoogleKnowledgeGraph](https://developers.google.com/knowledge-graph).

In [17]:
annotation_files=[os.getcwd() + '/assets/open_sample.csv']

df = pd.read_csv(annotation_files[0])
df.head(2)

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,0001eeaf4aed83f9,xclick,/m/0cmf2,1,0.022673,0.964201,0.071038,0.800546,0,0,0,0,0
1,000595fe6fee6369,xclick,/m/02wbm,1,0.000000,1.000000,0.000000,1.000000,0,0,1,0,0


When adding the file to the annotation set, remo automatically: 
- detects the class encoding and translates it into the corresponding labels
- only adds annotations for classes that are part of the existing annotation set

In [12]:
my_dataset.add_data(local_files=annotation_files, annotation_task = 'Object detection')

{'files_link_result': {'files uploaded': 0, 'annotations': 10, 'errors': []}}

In [13]:
my_dataset.get_annotation_statistics()

[{'AnnotationSet ID': 16,
  'AnnotationSet name': 'Objects',
  'n_images': 10,
  'n_classes': 18,
  'n_objects': 98,
  'top_3_classes': [{'name': 'Fruit', 'count': 27},
   {'name': 'Sports equipment', 'count': 12},
   {'name': 'Human arm', 'count': 10}],
  'creation_date': None,
  'last_modified_date': '2020-03-06T13:05:57.178032Z'}]

In [14]:
my_dataset.view()

Open http://localhost:8123/datasets/25


![dataset_added_annotation.jpeg](assets/dataset_added_annotation.jpeg) 

## Add annotations from code

**Let's see how we can add annotations to a specific image from code**

This can be useful for instance to add model predictions as annotations or tag specific images.

First, let's retrieve one image

In [18]:
images = my_dataset.images()
my_image = images[0]
print(my_image)
print('Resoultion: ', my_image.width, 'x', my_image.height)

Image: 925 - 000595fe6fee6369.jpg
Resoultion:  1024 x 681


**Now we can add annotations using the Annotation class**

We can add an annotation object to either the annotation set or directly to an image

In [76]:
annotation = remo.Annotation()
annotation.add_item(classes=['Person'], bbox=[100, 100, 300, 300])
annotation.add_item(classes=['Dog'], bbox=[250, 250, 400, 400])

annotation_set.add_annotation(my_image.id, annotation)

In [77]:
annotation = remo.Annotation()
annotation.add_item(classes=['Fruit'], bbox=[900, 500, 1000,650])

my_image.add_annotation(annotation)

In [78]:
my_image.view()

Open http://localhost:8123/image/905?dataset_id=23


![added_annotation.jpeg](assets/added_annotation.jpeg)

## TODO: Add annotations from custom file